In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TEMPERATURE = 0.7

In [4]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4', temperature=TEMPERATURE)
llm4_32k = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

"I am an AI language model based on OpenAI's GPT-3.5 Turbo."

In [5]:
llm3_5.model_name, llm4.model_name, llm4_32k.model_name

('gpt-3.5-turbo-16k', 'gpt-4', 'gpt-4-32k')

In [6]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
config = Config(
    ask_user_before_execute_codes=False,
    ask_user_questions_in_one_prompt=True,
    pause_after_initial_solving_expansion=True,
    first_expansion=1,
    initial_expansion=2,
    max_tokens=10000,
    check_final=True
)

primary_model = 3
sage_model = 3

if primary_model == 4:
    primary_model = LangChainLLM(llm4, logger=logger)
    sage_llm = primary_model
else:
    primary_model = LangChainLLM(llm3_5, logger=logger, long_context_llm=llm4_32k, long_context_token_threshold=3000)
    sage_model = LangChainLLM(llm4, logger=logger, long_context_llm=llm4_32k, long_context_token_threshold=3000) \
        if sage_model is None else primary_model
executor = Orchestrator(
    config=config,
    llm=primary_model,
    prompts=prompts,
    markdown_logger=logger,
    sage_llm=sage_model,
)

experiment_name = 'example'
executor.start("""What's x mod (y - z) where x = 10, y = 7, z = 4.""")
# executor.start("""
# Solve this 4x4 Sudoku.
#
# ```text
# _ 3 _ 1
# 1 _ _ 3
# 2 _ 3 _
# 3 4 _ 2
# ```
#
# (`_` represents an empty cell.)
#
# Solve in your head. Do NOT use Python programming.
# """)

Pause: Pause after initial solving expansion

In [7]:
# executor._prompts = PromptDb.load_defaults()

In [8]:
# executor._chain = executor._chain[:-1]
# executor._chain

In [9]:
# executor.chain[-1].description

In [10]:
executor.resume(10000, additional_tokens_for_smarter_llm=0, unblock_initial_expansion=True)

In [11]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
executor.log_configs(logger)
logger.log_conversation(executor.show_conversation_thread(with_header=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")


In [52]:
executor.chain

[PresentTaskStep(step_id=0,description=Solve this 4x4 Sudo,prev=None),
 AnalysisStep(step_id=1,description=The problem statemen,prev=0),
 ProceedStep([StepByStepPlan(step_id=3,description=```json
 {
   "1": {"d,prev=2,[int,int,int,int])],ptr=0),
 StepByStepPlan(step_id=3,description=```json
 {
   "1": {"d,prev=2,[int,int,int,int]),
 ProceedStep([DirectAnswerStep(step_id=5,description=To fill in the first,prev=4)],ptr=0),
 DirectAnswerStep(step_id=5,description=To fill in the first,prev=4),
 ProceedStep([DirectAnswerStep(step_id=7,description=To fill in the secon,prev=6)],ptr=0),
 DirectAnswerStep(step_id=7,description=To fill in the secon,prev=6),
 ProceedStep([DirectAnswerStep(step_id=9,description=To fill in the third,prev=8)],ptr=0),
 DirectAnswerStep(step_id=9,description=To fill in the third,prev=8),
 ProceedStep([DirectAnswerStep(step_id=10,description=To fill in the fourt,prev=10)],ptr=0),
 DirectAnswerStep(step_id=10,description=To fill in the fourt,prev=10),
 FinalAnswerStep(s

In [55]:
executor.chain[-2]

"To fill in the fourth row, we need to find the numbers that are not already present. The numbers already present in the fourth row are 3, 4, and 2. So, the missing number is 1.\n\nWe can't put 1 in the third cell because the third row already has a 1 in the third column. So, the third cell must be 1.\n\nThe fourth row now looks like this:\n\n```text\n3 4 1 2\n```\n\nNow, the entire Sudoku grid is filled:\n\n```text\n4 3 2 1\n1 2 4 3\n2 4 3 1\n3 4 1 2\n```"

In [57]:
step: DirectAnswerStep = executor.chain[-2]
step.is_final_step

False